### Imports

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz
import difflib 
from fuzzywuzzy import process
import time

### Importing inputs
- 3 Mout. Volumes 
- wcvp 
- adding a column with author lists to each dataframe

##### updating author functions

In [31]:
#TODO: move this to index code
def fix_author(authors_field):
    """this function removes the extra space that currently exists
       in the 'author' filed of the index csv files
       INPUT: any value
       OUTPUT: the function returns non string values as is
               and removes last character for string values 
    """
    if not isinstance(authors_field, str) or authors_field[-1] != ' ':
        return authors_field
    return authors_field[:-1]

In [32]:
#TODO: set the list of connectives as an external thing
def split_author(author_str):
    """function that splits the author names ignoring given connectives used
       INPUT: a string (of authors)
       OUTPUT: a list of containing each author from the input string
    """
    if (not isinstance(author_str, str)) or author_str == "np.NaN":
        return author_str
    paran_list = []
    reg_list = []

    in_pran = ""
    out_pran = ""
    #the paranthesis part (assuming we have one paranthesis only)
    search_full_paran = re.search(r"\(.*\)", author_str)
    search_close_paran = re.search(r".*\)", author_str)
    if search_full_paran:
        in_pran = search_full_paran.group(0)[1:-1] 
    elif search_close_paran:
        in_pran = search_close_paran.group(0)[:-1]
    
    split_paran = re.split(r".*\)", author_str, maxsplit=1)
    out_pran = split_paran[int(len(split_paran) == 2)]
    
    #remove anything after ex. 
    in_pran = re.sub(r"\sex[\s|.].*","",in_pran)
    out_pran = re.sub(r"\sex[\s|.].*","",out_pran)

    #ignoring et, in, non, &, er, nec, with space or . after (TODO: check end/ beggining of string)
    ignore_pattern = r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]|\ser[\s|.]|\snec[\s|.]|\smult[\s|.]"
    paran_list = re.split(ignore_pattern, in_pran) 
    reg_list = re.split(ignore_pattern, out_pran) 
    
    #TODO: improve this part
    return [aut for aut in paran_list + reg_list if aut != ""] 

In [33]:
# a function for flagging authors with auct. / agg. etc
# so that we can just use the wcvp version later
# note, vol3 only has 2 occurances of auct. 
# the rest don't show up because we are only looking at non-italicized part of index

##### importing dataframes 

In [34]:
#mout. books: 
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#changing name of columns of mout. indecies 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

##### applying the author functions

In [29]:
#fix spacing issue in Mout. books
vol1_df['mouterde_author'] = vol1_df['mouterde_author'].apply(fix_author)
vol2_df['mouterde_author'] = vol2_df['mouterde_author'].apply(fix_author)
vol3_df['mouterde_author'] = vol3_df['mouterde_author'].apply(fix_author)

#getting author lists
wcvp_df['author_list'] = wcvp_df['authors'].apply(split_author)
vol1_df['mouterde_author_list'] = vol1_df['mouterde_author'].apply(split_author)
vol2_df['mouterde_author_list'] = vol2_df['mouterde_author'].apply(split_author)
vol3_df['mouterde_author_list'] = vol3_df['mouterde_author'].apply(split_author)

### Finding closest Match for the genera

In [40]:
#uses default matching criteria of difflib
def difflib_closest_matches(input_str, matching_lst):
    matched_list = difflib.get_close_matches(input_str, matching_lst)
    if len(matched_list) > 0:
        return matched_list[0]
    else:
        print(input_str)
        return np.NaN

In [81]:
wcvp_unique_genera = wcvp_df['genus'].unique()


startTime = time.time()
#prints the ones with no closest match
print("---- vol1 genus match ----")
vol1_df['mout_genus_close_match'] = vol1_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol2 genus match ----")
vol2_df['mout_genus_close_match'] = vol2_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol3 genus match ----")
vol3_df['mout_genus_close_match'] = vol3_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

---- vol1 genus match ----
j.
j.
j.
j.
j.
j.
j.
j.
Seconds to Run:119.20491409301758
-------- complete --------

---- vol2 genus match ----
Seconds to Run:130.8245460987091
-------- complete --------

---- vol3 genus match ----
Seconds to Run:151.4979109764099
-------- complete --------



In [82]:
print(vol1_df[vol1_df['mout_genus_close_match'] != vol1_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol1_df[vol1_df['mout_genus_close_match'] != vol1_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

25


,mouterde_genus,mout_genus_close_match
31,AgropyrQn,Agropyron
214,Asphodelusi,Asphodelus
282,Bœrhavia,Boerhavia
320,Buffonia,Bufonia
372,Gerastium,Cerastium
383,Geterach,Ceterach
453,Çutandia,Cutandia
496,Danaë,Diana
544,Eremopyron,Eremophyton
618,filicaulis,Silicularia


In [83]:
print(vol2_df[vol2_df['mout_genus_close_match'] != vol2_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol2_df[vol2_df['mout_genus_close_match'] != vol2_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

27


,mouterde_genus,mout_genus_close_match
105,xanthocarpum,Acanthocarpus
118,Ampélopsis,Ampelopsis
143,Anisociadium,Anisosciadium
313,Àubrietia,Aubrieta
357,Buffonia,Bufonia
414,Cardariâ,Cardaria
436,•Ceratophyllum,Ceratophyllum
451,Ghesneya,Chesneya
460,Chrysocamela,Chrysochamela
466,Gircaea,Dircaea


In [84]:
print(vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

34


,mouterde_genus,mout_genus_close_match
71,Anthémis,Anthemis
77,corymbulosa,Corymbula
78,cotula,Rotula
79,eretica,Herpetica
82,didymaea,Didymaea
83,halophila,Halophila
84,haussknechtii,Haussknechtia
85,hebronica,Veronica
86,hyalina,Thyana
87,leucanthemifolia,Leucanthemella


In [85]:
# only the non-lower case ones for vol3: 
print(vol3_df[(vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']) & ~(vol3_df['mouterde_genus'].str.islower())][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']) & ~(vol3_df['mouterde_genus'].str.islower())][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

12


,mouterde_genus,mout_genus_close_match
71,Anthémis,Anthemis
157,BallotaL.,Ballota
209,Calhpeltis,Callipeltis
424,CordiaL.,Cordia
438,Crépis,Crepis
511,Flore,Floresia
559,Engeron,Erigeron
701,Hypochoeris,Hypochaeris
882,Moriria,Morierina
1188,Satureia,Saturnia


for volume3, use 'genus_matched' (includes both accepted matches and genera in vol3_accepted_genus_changes) to see the closest match of which genera should be used

In [107]:
vol1_accepted_genus_changes = {}
vol2_accepted_genus_changes = {}

vol3_accepted_genus_changes = {
    "Anthémis" : "Anthemis",
    "BallotaL." : "Ballota",
    "CordiaL." : "Cordia",
    "Crépis" : "Crepis",
    "SolidagoL." : "Solidago"
}

In [108]:
vol1_df['genus_matched'] = (vol1_df['mout_genus_close_match'] == vol1_df['mouterde_genus']) | (vol1_df['mouterde_genus'].isin(vol1_accepted_genus_changes))
vol2_df['genus_matched'] = (vol2_df['mout_genus_close_match'] == vol2_df['mouterde_genus']) | (vol2_df['mouterde_genus'].isin(vol2_accepted_genus_changes))
vol3_df['genus_matched'] = (vol3_df['mout_genus_close_match'] == vol3_df['mouterde_genus']) | (vol3_df['mouterde_genus'].isin(vol3_accepted_genus_changes))

### For each species, find the closes match within it's closest match genera

In [149]:
#uses default matching criteria of difflib
def difflib_closest_epithet_match(row):
    epithet = row['mouterde_epithet']
    if row['genus_matched'] and isinstance(epithet, str):
        genus = row['mout_genus_close_match']
        matching_list = wcvp_df[wcvp_df['genus'] == genus]['species'].unique()
        cleanedList = [x for x in matching_list if str(x) != 'nan']
        match_result = difflib.get_close_matches(epithet, cleanedList)  
        
        if match_result:
            return match_result[0]
        else:
            return np.NaN
    else: 
        return np.NaN

In [150]:
startTime = time.time()
#prints the ones with no closest match
print("---- vol1 epithet match ----")
vol1_df['mout_epithet_close_match'] = vol1_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

startTime = time.time()
print("---- vol2 epithet match ----")
vol2_df['mout_epithet_close_match'] = vol2_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

startTime = time.time()
print("---- vol3 epithet match ----")
vol3_df['mout_epithet_close_match'] = vol3_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

---- vol1 epithet match ----
Seconds to Run:36.77819514274597
--------- complete ---------

---- vol2 epithet match ----
Seconds to Run:44.84345507621765
--------- complete ---------

---- vol3 epithet match ----
Seconds to Run:51.42594790458679
--------- complete ---------



##### Places where non NaN matches of the epithet do not overlap and the number of such occurances

In [172]:
print(vol1_df[(vol1_df['mout_epithet_close_match'] != vol1_df['mouterde_epithet']) & ~(vol1_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol1_df[(vol1_df['mout_epithet_close_match'] != vol1_df['mouterde_epithet']) & ~(vol1_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

91


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
12,Aegilops,juvenali∫,juvenalis
17,Aegilops,peregrina(Hack.),peregrina
22,Aegilops,umbeUulata,umbellulata
27,Aellenia,glauc,glauca
71,Allium,de∫er˛i,deserti
...,...,...,...
1268,Spergularia,bocconii,bocconei
1279,Spirodela,polyrrhiza,polyrhiza
1328,Trisetaria,kœlerioides,koelerioides
1336,Tulipa,auCheriana,aucheriana


In [173]:
print(vol2_df[(vol2_df['mout_epithet_close_match'] != vol2_df['mouterde_epithet']) & ~(vol2_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol2_df[(vol2_df['mout_epithet_close_match'] != vol2_df['mouterde_epithet']) & ~(vol2_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

107


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
6,Acer,tauricolum,tauricum
58,Alchemilla,noribunda,floribunda
62,Alhagi,pseudalhâgi,pseudalhagi
74,Althaea,bertrami,bertramii
98,Alyssum,stapffii,stapfii
...,...,...,...
1389,Polypogon,semiyerticillatus,semiverticillatus
1425,Psammogeton,setifolium,setifolius
1432,Pteris,vïttata,vittata
1451,Ranunculus,macrorrhynchup,macrorrhynchus


In [174]:
print(vol3_df[(vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']) & ~(vol3_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']) & ~(vol3_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

41


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
126,Artemisia,verlotorum,verlotiorum
156,Atropa,belladonna,bella-donna
320,Centaurea,ptosimoppapa,ptosimopappa
347,Centranthus,longinorus,longiflorus
462,Crepis,zacyntha,zacintha
464,Cressa,eretica,cretica
582,Fontanesia,phillyreoides,philliraeoides
640,Gnaphalium,luteo-album,luteoalbum
651,Hedypnois,eretica,cretica
683,Hieracium,bauhinii,bauhini


##### Closest match for genus exists
##### but closest match for epithet doesn't

In [175]:
vol1_df[~(vol1_df['mouterde_epithet'].isna()) & (vol1_df['mout_epithet_close_match'].isna()) & (vol1_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
25,Aegilotricum,loreti,NaN
550,Eremurus,hostii,NaN
551,Eremurus,ravennae,NaN
650,Heleocharis,macrantha,NaN
651,Heleocharis,palustris,NaN
698,Hypocylix,kerneri,NaN
1267,Spergularia,et,NaN


In [176]:
vol2_df[~(vol2_df['mouterde_epithet'].isna()) & (vol2_df['mout_epithet_close_match'].isna()) & (vol2_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
393,Callitriche,sp.,NaN
396,Calycotome,villosa,NaN
433,Ceratocephalus,falcafus,NaN
504,Consolida,flâva,NaN
667,Erucaria,boveana,NaN
982,Iberis,sofarana,NaN
1069,Linum,corymbulosum(Planch.)Reichenb.,NaN
1074,Linum,sulphureurn,NaN


In [177]:
vol3_df[~(vol3_df['mouterde_epithet'].isna()) & (vol3_df['mout_epithet_close_match'].isna()) & (vol3_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
631,Garhadiolus,et,NaN
689,Hormuzakia,aggregata,NaN
888,Myopordon,genitz,NaN
930,Odontites,hitea,NaN
1038,Phyllocara,aucheri,NaN
1097,Psychrogeton,son,NaN


## Untitled